In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("/home/prasun/GitDemo/Time_Trekking/Dataset/book_sales.csv",
                 index_col="Date", parse_dates=["Date"]).drop(["Paperback"],axis="columns")
df.head()

,Hardcover
Date,
2000-04-01,139
2000-04-02,128
2000-04-03,172
2000-04-04,139
2000-04-05,191
